In [1]:
import os
import pandas as pd
from dotenv import load_dotenv
from langchain_core.runnables import RunnableConfig
from langchain_teddynote.messages import random_uuid, invoke_graph

import sys
sys.path.append("../../")

from graph import DataExtractor

ModuleNotFoundError: No module named 'graph'

In [2]:
# .env 파일 로드
load_dotenv(dotenv_path=".env")

# API 키 가져오기
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")

# LangSmith 추적 기능을 활성화합니다. (선택적)
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Multi-agent Collaboration"

In [3]:
voltai_graph = DataExtractor(
    folder_path="/home/aiffel04/AIFFEL/VOLTAI/data/papers/", 
    file_number=33
).graph

In [4]:
# config 설정(재귀 최대 횟수, thread_id)
config = RunnableConfig(recursion_limit=20, configurable={"thread_id": random_uuid()})

In [5]:
# 4개의 질문 입력
questions = [
        """Below are instructions for filling out items by referring to the examples.
    The values shown to the right of the colon (“:”) are examples;
    please delete them after reviewing and rewrite them with the values found in the PDF.
    If any item is not mentioned in the PDF, do not remove it—write “None.”
    [
        {
             "CAM (Cathode Active Material)": {
                "Stoichiometry information": {"NCM-622": {"Li ratio": "1", "Ni ratio": "0.6", "Co ratio": "0.2", "Mn ratio": "0.2", "O ratio": "2"}, "ZrO2-NCM-622 (Z622)": {"Li ratio": "0.98", "Ni ratio": "0.6", "Co ratio": "0.2", "Mn ratio": "0.2", "O ratio": "2"}},
                "Whether or not commercial NCM was used for each sample (Stoichiometry information in order)": ["yes", "no"],
                "Lithium source": "LiOH",
                "Synthesis method": "co-precipitation",
                "Describe the crystallization method, such as Hydrothermal, Sintering, or any other technique used during the process.": "Hydrothermal",
                "What is the Crystallization final temperature in degree of Celcius used in the process? (e.g., calcination or sintering) mentioned for the crystallization stage.": "100",
                "What is the time duration for the final crystallization process, including any calcination or sintering stages? Specify the hours.": "12",
                "Doping": "Zr4+",
                "Coating": "ZrO2",
                "Additional treatment": "None" }
        }
    ]
    """,    
    
    """Below are instructions for filling out items by referring to the examples.
    The values shown to the right of the colon (“:”) are examples;
    please delete them after reviewing and rewrite them with the values found in the PDF.
    If any item is not mentioned in the PDF, do not remove it—write “None.”
    
    [
        {
            "Electrode (only for coin-cell (half-cell))": {
                "Active material : Conductive additive : Binder ratio": "90 : 5 : 5",
                "Electrolyte": {"Salt":"LiPF6", "Concentration":"1M", "Solvent":"EC:EMC:DEC", "Solvent ratio":"1:1:1"},
                "Additive": "FEC 10% addition",
                "Loading density (mass loading of NCM, Standardize the unit to mg/cm^2)": "5",
                },
        }
    ]
    """,

    """Below are instructions for filling out items by referring to the examples.
    The values shown to the right of the colon (“:”) are examples;
    please delete them after reviewing and rewrite them with the values found in the PDF.
    If any item is not mentioned in the PDF, do not remove it—write “None.”
    Provide a detailed explanation of the SEM and TEM results, ensuring all relevant information is included even if dispersed across multiple sections or chunks. Mention particle morphology, size, or any other characteristics noted in the results.
    [
        {        
            "Morphological results": {
                "Explanation of SEM results": "Fig. 2a, b; the NCM-622 seems to have more or less a spherical morphology with a diameter of 3–5 µm, composed of densely packed primary particles",
                "Explanation of TEM results": "In Figure 2 (a) and (b), the (002) plane is observed, with a d-spacing of 0.472 nm. Additionally, a layered structure is evident."
                },    
        }
    ]
    """,

    """Below are instructions for filling out items by referring to the examples.
    The values shown to the right of the colon (“:”) are examples;
    please delete them after reviewing and rewrite them with the values found in the PDF.
    If any item is not mentioned in the PDF, do not remove it—write “None.”
    List all discharge capacities (mAh/g) at different conditions (e.g., initial capacity, after cycling, different C-rates, cutoff voltages). Include the electrode state and any specified conditions such as current density, cycling stage, or voltage range.
    List all discharge capacities (mAh/g) for pristine, doped, and coated electrodes at all specified C-rates (e.g., 0.1C, 5.0C). Include electrode state (e.g., pristine, doped, coated) and any relevant conditions or performance metrics.
    [
        {            
            "Cathode Performance": {
                "Discharge Capacity": [{
                "200.2 mAh/g": "@0.1C, Pristine",
                "191.3 mAh/g": "@0.5C, Pristine",
                "214.5 mAh/g": "@0.1C, ZrO2-coated",
                "200.8 mAh/g": "@0.5C, ZrO2-coated"
                }],
                "Voltage range": "2.8–4.3 V",
                "Temperature": "Room temperature and 55°C"
                }
        }
    ]
    """
]


In [6]:
# 4개의 질문에 대해 그래프 실행 및 출력
outputs = []
for i, question in enumerate(questions):
    # inputs = GraphState(question=question)
    print(f"#####   {i}번째 질문   #####")
    # print(question)
    result = voltai_graph.invoke(
        input={"question":question},
        config=config,
    )
    outputs.append(result)

#####   0번째 질문   #####
==== [RELEVANCE CHECK] ====
yes
#####   1번째 질문   #####
==== [RELEVANCE CHECK] ====
yes
#####   2번째 질문   #####
==== [RELEVANCE CHECK] ====
yes
#####   3번째 질문   #####
==== [RELEVANCE CHECK] ====
yes


In [8]:
# 1번 question 최적화를 위한 코드
import pprint
outputs_dict = outputs[0]["answer"][0]["CAM (Cathode Active Material)"]
pprint.pprint(outputs_dict)

# 2번 결과 출력 코드
#import pprint
#outputs_dict = outputs[0]["answer"][0]["Electrode (only for coin-cell (half-cell))"] 
#pprint.pprint(outputs_dict)

# 3번 결과 출력 코드
#import pprint
#outputs_dict = outputs[0]["answer"][0]["Morphological results"] 
#pprint.pprint(outputs_dict)

# 4번 결과 출력 코드
#import pprint
#outputs_dict = outputs[0]["answer"][0]["Cathode Performance"] 
#pprint.pprint(outputs_dict)

{'Additional treatment': 'None',
 'Coating': 'None',
 'Describe the crystallization method, such as Hydrothermal, Sintering, or any other technique used during the process.': 'None',
 'Doping': 'None',
 'Lithium source': 'None',
 'Stoichiometry information': {'NCM-622': {'Co ratio': '0.2',
                                           'Li ratio': '1.02',
                                           'Mn ratio': '0.2',
                                           'Ni ratio': '0.6',
                                           'O ratio': '2'},
                               'ZrO2-NCM-622 (Z622)': 'None'},
 'Synthesis method': 'None',
 'What is the Crystallization final temperature in degree of Celcius used in the process? (e.g., calcination or sintering) mentioned for the crystallization stage.': 'None',
 'What is the time duration for the final crystallization process, including any calcination or sintering stages? Specify the hours.': 'None',
 'Whether or not commercial NCM was used for each sam

In [7]:
import pprint
outputs_dict = outputs[0]["answer"][0]["CAM (Cathode Active Material)"] | outputs[1]["answer"][0]["Electrode (only for coin-cell (half-cell))"] | outputs[2]["answer"][0]["Morphological results"] | outputs[3]["answer"][0]["Cathode Performance"]
pprint.pprint(outputs_dict)

{'Active material : Conductive additive : Binder ratio': '90 : 5 : 5',
 'Additional treatment': 'Annealing at 600°C for 8 hours',
 'Additive': 'None',
 'Coating': 'Al2O3/LiAlO2',
 'Describe the crystallization method, such as Hydrothermal, Sintering, or any other technique used during the process.': 'None',
 'Discharge Capacity': [{'153.5 mAh/g': '@0.1C, Coated (C-NCM-600)',
                         '58.9 mAh/g': 'after 190 cycles @0.5C, Pristine '
                                       '(P-NCM)',
                         'None': '@4C, Coated (C-NCM-600)'}],
 'Doping': 'None',
 'Electrolyte': 'LP50 (50μL, Sigma-Aldrich)',
 'Explanation of SEM results': 'The SEM images of the NCM701515 powders (Fig. '
                               '2a-c) show that both pristine and coated '
                               'samples have a spherical morphology with '
                               'secondary particles having a diameter of 5-15 '
                               'μm. These secondary particle

In [8]:
# 알파벳순 말고 그대로 출력하는 코드
import json

print(json.dumps(outputs_dict, indent=4, ensure_ascii=False))


{
    "Stoichiometry information": {
        "NCM701515": {
            "Li ratio": "1",
            "Ni ratio": "0.7",
            "Co ratio": "0.15",
            "Mn ratio": "0.15",
            "O ratio": "2"
        }
    },
    "Whether or not commercial NCM was used for each sample (Stoichiometry information in order)": [
        "yes"
    ],
    "Lithium source": "None",
    "Synthesis method": "None",
    "Describe the crystallization method, such as Hydrothermal, Sintering, or any other technique used during the process.": "None",
    "What is the Crystallization final temperature in degree of Celcius used in the process? (e.g., calcination or sintering) mentioned for the crystallization stage.": "None",
    "What is the time duration for the final crystallization process, including any calcination or sintering stages? Specify the hours.": "None",
    "Doping": "None",
    "Coating": "Al2O3/LiAlO2",
    "Additional treatment": "Annealing at 600°C for 8 hours",
    "Active mater

In [9]:
pd.DataFrame(outputs_dict)

,Stoichiometry information,Commercial NCM,Lithium source,Synthesis method,"Describe the crystallization method, such as Hydrothermal, Sintering, or any other technique used during the process.","What is the crystallization temperature used in the process? Specify all related temperatures (e.g., calcination or sintering) mentioned for the crystallization stage.","What is the time duration for the crystallization process, including any calcination or sintering stages? Specify the hours or minutes mentioned in the experimental procedure.",Doping,Coating,Additional treatment,...,Additive,Electrode thickness,Only Cathode Electrode diameter,Loading density (mass loading of NCM),Additional treatment for electrode,Explanation of SEM results,Explanation of TEM results,Discharge Capacity,Voltage range,Temperature
0,LiNi0.8Co0.1Mn0.1O2,Yes,LiOH·H2O,co-precipitation,Sintering,"500°C for calcination, 750°C for sintering","5 hours for calcination, 15 hours for sintering",Mg2+ doping,Li3PO4 coating,None,...,None,None,None,None,None,"The SEM images (Figure 2A, B) show that both p...","The TEM images (Figure 2C, D) reveal the prese...","{'199.1 mAh/g': '@0.1C, Pristine'}",3.0–4.3 V,Room temperature (25°C)
1,LiNi0.8Co0.1Mn0.1O2,Yes,LiOH·H2O,co-precipitation,Sintering,"500°C for calcination, 750°C for sintering","5 hours for calcination, 15 hours for sintering",Mg2+ doping,Li3PO4 coating,None,...,None,None,None,None,None,"The SEM images (Figure 2A, B) show that both p...","The TEM images (Figure 2C, D) reveal the prese...","{'203.5 mAh/g': '@0.1C, M-NCM (Mg-doped, Li3PO...",3.0–4.3 V,Room temperature (25°C)
2,LiNi0.8Co0.1Mn0.1O2,Yes,LiOH·H2O,co-precipitation,Sintering,"500°C for calcination, 750°C for sintering","5 hours for calcination, 15 hours for sintering",Mg2+ doping,Li3PO4 coating,None,...,None,None,None,None,None,"The SEM images (Figure 2A, B) show that both p...","The TEM images (Figure 2C, D) reveal the prese...","{'103.3 mAh/g': '@1.0C, Pristine'}",3.0–4.3 V,Room temperature (25°C)
3,LiNi0.8Co0.1Mn0.1O2,Yes,LiOH·H2O,co-precipitation,Sintering,"500°C for calcination, 750°C for sintering","5 hours for calcination, 15 hours for sintering",Mg2+ doping,Li3PO4 coating,None,...,None,None,None,None,None,"The SEM images (Figure 2A, B) show that both p...","The TEM images (Figure 2C, D) reveal the prese...","{'143.6 mAh/g': '@1.0C, M-NCM (Mg-doped, Li3PO...",3.0–4.3 V,Room temperature (25°C)
4,LiNi0.8Co0.1Mn0.1O2,Yes,LiOH·H2O,co-precipitation,Sintering,"500°C for calcination, 750°C for sintering","5 hours for calcination, 15 hours for sintering",Mg2+ doping,Li3PO4 coating,None,...,None,None,None,None,None,"The SEM images (Figure 2A, B) show that both p...","The TEM images (Figure 2C, D) reveal the prese...","{'159.0 mAh/g': '@5.0C, Pristine'}",3.0–4.3 V,Room temperature (25°C)
5,LiNi0.8Co0.1Mn0.1O2,Yes,LiOH·H2O,co-precipitation,Sintering,"500°C for calcination, 750°C for sintering","5 hours for calcination, 15 hours for sintering",Mg2+ doping,Li3PO4 coating,None,...,None,None,None,None,None,"The SEM images (Figure 2A, B) show that both p...","The TEM images (Figure 2C, D) reveal the prese...","{'180.0 mAh/g': '@5.0C, M-NCM (Mg-doped, Li3PO...",3.0–4.3 V,Room temperature (25°C)
6,LiNi0.8Co0.1Mn0.1O2,Yes,LiOH·H2O,co-precipitation,Sintering,"500°C for calcination, 750°C for sintering","5 hours for calcination, 15 hours for sintering",Mg2+ doping,Li3PO4 coating,None,...,None,None,None,None,None,"The SEM images (Figure 2A, B) show that both p...","The TEM images (Figure 2C, D) reveal the prese...","{'169.8 mAh/g': '@4.0C, Pristine'}",3.0–4.3 V,Room temperature (25°C)
7,LiNi0.8Co0.1Mn0.1O2,Yes,LiOH·H2O,co-precipitation,Sintering,"500°C for calcination, 750°C for sintering","5 hours for calcination, 15 hours for sintering",Mg2+ doping,Li3PO4 coating,None,...,None,None,None,None,None,"The SEM images (Figure 2A, B) show that both p...","The TEM images (Figure 2C, D) reveal the prese...","{'168.9 mAh/g': '@6.0C, M-NCM (Mg-doped, Li3PO...",3.0–4.3 V,Room 